# ROOF RECONSTRUCTION ENRICHMENT

## ADD LIBRARIES

In [1]:

#IMPORT PACKAGES
import rdflib
from rdflib import Graph, URIRef,Namespace, Literal, OWL,RDFS, RDF, XSD,RDFS

import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
import ifcopenshell.util.selector
import random as rd
import pandas as pd
import cv2
from pathlib import Path
import pye57
import laspy
import copy
import datetime
import ezdxf 
from PIL import Image, ImageDraw, ImageFont
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from collections import Counter
import json
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.utils.cadutils as cadu
import geomapi.utils.imageutils as iu
import geomapi.tools as tl
import sys


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## ADD DATA

In [2]:
#path
path=Path('V:/Studenten/Thesis/Eline Deblock/data/hoekpand')

#images
panoPath = path / 'WE973UB5.jfif'
pano = Image.open(panoPath)
#convert to numpy array
camJsonPath = path /'cam_json.json'
with open(camJsonPath) as f:
    camJson = json.load(f)

#pointcloud -> this was already geolocated
lasPath = path / "street_lidar.laz"
las = laspy.read(lasPath)
pcd=gmu.las_to_pcd(las)



#mesh
meshPath = path / "referentie_mesh_remapped_z.obj" #this mesh was resaved with Rhino7 to remap the Y to Z axis 
mesh = o3d.io.read_triangle_mesh(str(meshPath))
meshRelocation=np.array([[-0.559193, -0.829038, 0.000000 ,19.037735],
                            [0.829038, -0.559193, 0.000000, 27.333549],
                            [0.000000 ,0.000000, 1.000000, 20.909252],
                            [0.000000, 0.000000, 0.000000, 1.000000]]) # this relocation was performed in CC
mesh.transform(meshRelocation)
offsetX=72800.00
offsetY=167550.00
mesh.translate([offsetX,offsetY,0])

#cityjson
cityjsonPath = path / "cityjson.json"
with open(cityjsonPath) as f:
    cityjson = json.load(f)
    cityObjects = cityjson.get("CityObjects", {})


c:\Users\u0094523\.conda\envs\geomapi_dev\lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (103680000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [3]:
#show mesh and pcd
o3d.visualization.draw_geometries([mesh,pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


## PARSE CITYJSON 


This parsing is not so straightfoward because meshes can both contain triangle (3 vertices) and multisurface (n vertices) geometries.
However, this geometry is optional if you have the Cloucompare obj meshes which are already triangulated.

In [ ]:
# from scipy.spatial import Delaunay
# from shapely.geometry import Polygon, MultiPolygon
# from shapely.ops import triangulate

# #get all vertices
# vertices = np.array(cityjson.get("vertices", []))

# #get all objects
# objects = []
# for obj_id, obj_data in city_objects.items():
#     geometries = obj_data.get("geometry")[0]
#     # print(geometries)
#     if geometries.get("lod") == "1.2": #let's take lod1.2 for now
#         print(geometries)
#         vertex_counter = Counter()
#         boundaries=geometries.get("boundaries")[0]
#         for boundary in boundaries:
#             boundary = boundary[0]
#             points = np.array([vertices[i] for i in boundary])
#             print(boundary)

#             if len(boundary)==3: #triangle
#                 #increase counter by 3
#                 vertex_counter.update(boundary)
#                 #get triangle based on counter values
#                 points = np.array([vertices[i] for i in vertex_counter.keys()])
                
#             if len(boundary)>3: #multisurface
#                 print('multi_surface')
#                 #triangulate the multisurface using schipy delaynay triangulation

#         print(boundaries[0])

instead, just use the mesh and select the proper city_object with an lod1.2 geometry


In [5]:

for obj_id, obj_data in cityObjects.items():
    geometries = obj_data.get("geometry")[0]
    if geometries.get("lod") == "1.2": #let's take lod1.2 for now
        print('object_id: '+obj_id)
        city_object=cityObjects[obj_id]
        #print city_object attributes per line
        for key, value in city_object.items():
            print(key, ' : ', value)
        break


object_id: 1-0
attributes  :  {}
geometry  :  [{'boundaries': [[[[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]], [[70, 28, 27, 71]], [[72, 55, 54, 73]], [[73, 54, 53, 74]], [[75, 29, 28, 70]], [[76, 24, 23, 77]], [[78, 30, 29, 75]], [[79, 26, 25, 80]], [[81, 15, 14, 82]], [[83, 65, 64, 84]], [[85, 20, 19, 86]], [[71, 27, 26, 79]], [[87, 69, 68, 88]], [[89, 32, 31, 90]], [[80, 25, 24, 76]], [[91, 59, 58, 92]], [[93, 57, 56, 94]], [[94, 56, 55, 72]], [[84, 64, 63, 95]], [[96, 62, 61, 97]], [[98, 45, 44, 99]], [[99, 44, 43, 100]], [[100, 43, 42, 101]], [[101, 42, 41, 102]], [[103, 10, 69, 87]], [[104, 48, 47, 105]], [[92, 58, 57, 93]], [[106, 12, 11, 107]], [[108, 16, 15, 81]], [[82, 14, 13, 109]], [[110, 33, 32, 89]], [[111, 38, 37, 112]], [[86, 19, 18, 113]], [[114, 52, 51, 115]], [[74, 53, 52, 

## PARSE IMAGES

let's parse only 1 image

In [6]:
for imgId, properties in camJson.items():
    #Let's create an ImageNode
    
    # for key, value in properties.items():
        # print(key, ' : ', value)
        #add these values to the imgNode object
        # setattr(imgNode,key,value)
        
    #create rotation matrix from the yaw+heading with the heading in the direction of the X axis
    #rotation matrix
    # R_yaw = R.from_euler('z', properties.get('Yaw')+properties.get('Heading'), degrees=True).as_matrix()
    # R_roll = R.from_euler('y', -properties.get('Yaw'), degrees=True).as_matrix()
    
    
    R_yaw = R.from_euler('z',0, degrees=True).as_matrix() # properties.get('Heading')
    R_roll = R.from_euler('y', +0.85, degrees=True).as_matrix()  #90 #0
    R_pitch = R.from_euler('x', 180, degrees=True).as_matrix() #90 #90
    
    R_total = R_yaw @ R_roll @ R_pitch 
    
    
    # R_total = R_yaw #transpose the matrix to get the correct orientation
    #get cartesian transform (4x4) from the translation and rotation matrix
    cartesianTransform=gmu.get_cartesian_transform(translation=[properties.get('X'),properties.get('Y'),properties.get('Z')+2.07],rotation=R_total)

    #create a PanoNode with all these properties
    imgNode=PanoNode(name=imgId,resource=pano,cartesianTransform=cartesianTransform)  
    imgNode.accuracy=0.01
    imgNode.coordinaatSysteem=properties.get('Coordinaatsysteem')
    imgNode.hoogteSysteem=properties.get('Hoogtesysteem')
    imgNode.cameraHoogte=properties.get('Camerahoogte')
    
    #run these functions to get the image width and height -> these were not updated for some reason
    imgNode.imageWidth
    imgNode.imageHeight

#print imgNode attributes
print(imgNode.__dict__)

{'_depthPath': None, '_depthMap': None, '_imageWidth': 14400, '_imageHeight': 7200, '_jsonPath': None, '_resource': array([[[254, 255, 255],
        [254, 255, 255],
        [254, 255, 255],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [254, 255, 255]],

       [[254, 255, 255],
        [254, 255, 255],
        [254, 255, 255],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [254, 255, 255]],

       [[254, 255, 255],
        [254, 255, 255],
        [254, 255, 255],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [254, 255, 255]],

       ...,

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 12

let's project some rays to evaluate the heading

In [8]:
rays=imgNode.create_rays()
lineset=gmu.rays_to_lineset(rays)
#color first line green
colors=np.zeros((6,3))
colors[0]=[0,1,0] 
lineset.colors=o3d.cpu.pybind.utility.Vector3dVector(colors)
imgNode.lineSet=lineset
# show image cone, mesh and pcd and lineset
o3d.visualization.draw_geometries([mesh,pcd,imgNode.convexHull,lineset])

## DETECT WINDOWS

We will use the huggingface Grounding Dino model.
This is a zero shot model for image object detection. (https://huggingface.co/IDEA-Research/grounding-dino-base)
Just change the text prompt and image and you are good to go:

```
text = "window"
image_url = pano
```

You can install this in your environment simply by running 
```pip install transformers```

Note that you also need Pytorch: 
```pip install torch```



In [11]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection 

model_id = "IDEA-Research/grounding-dino-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

# VERY important: text queries need to be lowercased + end with a dot
image=pano
text = "window. door."

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

results = processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4, # play with this for better results
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)
print(results)

[{'scores': tensor([0.4100, 0.4536, 0.4006, 0.4355, 0.4452, 0.4086, 0.4181, 0.4072, 0.4599]), 'labels': ['window', 'window', 'window', 'window', 'window', 'window', 'window', 'window', 'window'], 'boxes': tensor([[9089.0918, 3235.0491, 9320.1855, 3414.4900],
        [3751.7937, 3403.9133, 3867.2180, 3763.0483],
        [8757.9199, 3248.3188, 8980.5938, 3419.9548],
        [4545.5972, 2701.5757, 4653.1421, 3103.4363],
        [3447.8989, 3420.6023, 3613.4351, 3758.9954],
        [4793.8242, 2834.5500, 4904.4473, 3200.8601],
        [3427.6785, 2980.7693, 3513.9539, 3264.6577],
        [3550.6790, 2941.5901, 3637.2947, 3242.9526],
        [3999.3037, 3381.9260, 4111.9180, 3781.9495]])}]


In [12]:
# Draw boxes around objects and add labels and scores
# image = image.convert("RGB") #maybe not necessary

#deepcopy pano
image = pano.copy()
draw = ImageDraw.Draw(image)

# Extract data from results
for result in results:
    scores = result['scores'].tolist()
    labels = result['labels']
    boxes = result['boxes'].tolist()

    # Loop through detections and draw each box
    for score, label, box in zip(scores, labels, boxes):
        # Convert box to int (Pillow requires integer coordinates)
        box = [int(coord) for coord in box]

        # Draw the rectangle
        draw.rectangle(box, outline="red", width=3)

        # Add the score as text (column,row)
        draw.text((box[0], box[1] - 10), f"{score:.2f}", fill="red")
    
# Display or save the resulting image
image.show()
        # break
# image.save("annotated_image.jpg")  # Uncomment to save the image

## REPROJECT DETECTIONS ON MESH

In [13]:
boxes = result['boxes'].tolist()
linesets=[]
for box in boxes:
    print(box)
    #convert box to 4 corner points
    # boxPoints=np.array([[box[1],box[0]], #xmin,ymin -> topleft
    #                     [box[1],box[2]], #xmax,ymin -> topright
    #                     [box[3],box[0]], #xmin,ymax -> bottomleft
    #                     [box[3],box[2]]]) #xmax,ymax   -> bottomright
    boxPoints=np.array([[box[0],box[1]], #xmin,ymin -> topleft
                        [box[2],box[1]], #xmax,ymin -> topright
                        [box[0],box[3]], #xmin,ymax -> bottomleft
                        [box[2],box[3]]]) #xmax,ymax   -> bottomright
    
    #sort to row, column
    rays=imgNode.create_rays(boxPoints,depths=50)
    lineset=gmu.rays_to_lineset(rays)
    #paint lineset red
    lineset.paint_uniform_color([1, 0, 0])
    linesets.append(lineset)  
    # break
print(linesets)  

[9089.091796875, 3235.049072265625, 9320.185546875, 3414.489990234375]
[3751.793701171875, 3403.913330078125, 3867.218017578125, 3763.04833984375]
[8757.919921875, 3248.31884765625, 8980.59375, 3419.954833984375]
[4545.59716796875, 2701.57568359375, 4653.14208984375, 3103.436279296875]
[3447.89892578125, 3420.602294921875, 3613.43505859375, 3758.995361328125]
[4793.82421875, 2834.550048828125, 4904.447265625, 3200.860107421875]
[3427.678466796875, 2980.769287109375, 3513.953857421875, 3264.65771484375]
[3550.678955078125, 2941.590087890625, 3637.294677734375, 3242.95263671875]
[3999.3037109375, 3381.926025390625, 4111.91796875, 3781.949462890625]
[LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines., LineSet with 4 lines.]


In [14]:
#show image cone, mesh and pcd and lineset
linesetsCombined=gmu.join_geometries(linesets)
o3d.visualization.draw_geometries([mesh,pcd,imgNode.convexHull,imgNode.lineSet,linesetsCombined])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [ ]:

lineset.transform(cartesianTransform)


In [3]:
import subprocess

input_file = path/ Path("cityjson.json")
output_file = "test.gml"

# Command to convert CityJSON to CityGML
command = ["cjio", input_file, "export", "--format", "citygml"]

# Execute the command and write to output file
with open(output_file, "w") as out:
    subprocess.run(command, stdout=out)

print(f"Converted {input_file} to {output_file} successfully.")


Converted V:\Studenten\Thesis\Eline Deblock\data\hoekpand\cityjson.json to test.gml successfully.


In [ ]:
citygml-tools export-geometry -i test.gml -o output.obj --triangulate


In [11]:
input_gml

WindowsPath('test.gml')

In [ ]:
import subprocess
from pathlib import Path

# Define input and output paths using Path from pathlib
input_gml = path/Path("test.gml")
output_obj = Path("test.obj")

# Command to convert CityGML to OBJ using citygml-tools
command = [
    "citygml-tools", "export-geometry",
    "-i", str(input_gml),  # Convert Path object to string
    "-o", str(output_obj),  # Convert Path object to string
    "--triangulate"
]

# Run the command
try:
    subprocess.run(command, check=True)
    print(f"Converted {input_gml} to {output_obj} successfully.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")


FileNotFoundError: [WinError 2] The system cannot find the file specified